In [ ]:
from tvb.simulator.models.oscillator import Generic2dOscillator
from tvb.simulator.integrators import HeunStochastic
from tvb.simulator.simulator import Simulator
from tvb.simulator.monitors import Raw
from tvb.datatypes.connectivity import Connectivity
from tvb.simulator.backend.theano import TheanoBackend

import matplotlib.pyplot as plt
import numpy as np
import theano.tensor as tt
import theano
import math
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# create simulator with oscillator model
osc = Generic2dOscillator()
conn = Connectivity.from_file()
conn.speed = np.r_[3.0]
integrator = HeunStochastic(dt=0.01)

sim = Simulator(
    connectivity=conn,
    model=osc,
    integrator=integrator,
    monitors=[Raw()],
    simulation_length=0.1
)
sim.configure()

In [ ]:
buf = sim.history.buffer[...,0]
rbuf = np.concatenate((buf[0:1], buf[1:][::-1]), axis=0)
state = np.transpose(rbuf, (1, 0, 2)).astype('f')

In [ ]:
# run simulation
(t, X), = sim.run()

In [ ]:
# generate code with theano backend
template = '<%include file="theano-sim.py.mako"/>'
kernel = TheanoBackend().build_py_func(template, dict(sim=sim, np=np), print_source=True)

In [ ]:
# define arguments for kernel
n_svar, _, n_node = state.shape

state = tt.as_tensor_variable(state.copy(), name="state")
state = state.reshape((n_svar, sim.connectivity.horizon, n_node))

weights = tt.as_tensor_variable(sim.connectivity.weights.copy(), name="weights")

Xh = tt.zeros((len(t), n_svar, n_node))

parmat = sim.model.spatial_parameter_matrix

In [ ]:
# run simulation with generated code for theano
np.random.seed(42)
args = state, weights, Xh, parmat, integrator.noise.nsig, sim.connectivity.delay_indices
Xh = kernel(*args)

In [ ]:
Xh.eval()